In [1]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
from tqdm import tqdm

from registration_metrics import CARLA_dataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
NOT_PLANES = 0
PLANES = 1
GROUND = 2
NOT_SURE = 3
UNKNOWN = 4


def get_CARLA_map_labels():
    not_planes_labels = [5, 9, 10, 18, 19, 20]
    planes_labels = [1, 2, 11]
    ground_labels = [0, 7, 8, 14]
    not_sure_labels = [12]
    unknown_labels = [3, 4, 6, 13, 15, 16, 17]

    map_labels = -np.ones(21, dtype=int)
    for labels, mark in zip(
        [not_planes_labels, planes_labels, ground_labels, not_sure_labels, unknown_labels],
        [NOT_PLANES, PLANES, GROUND, NOT_SURE, UNKNOWN]
    ):
        map_labels[labels] = mark

    return map_labels

## Приведение CARLA к формату SemanticKITTI

- папка с названием последовательности
- внутри папка velodyne с облаками в формате bin и labels c метками в формате npy

In [5]:
data = CARLA_dataset(start_id=0, end_id=2421, dataset_coords_type="world")

In [6]:
data_dir = '/mnt/f/Projects data, datasets/Goldobina/Research/carla_semantic/seq5_kitti_fmt/'
cld_dir = 'velodyne'
label_dir = 'labels'

In [32]:
os.makedirs(os.path.join(data_dir, cld_dir))
os.makedirs(os.path.join(data_dir, label_dir))

In [7]:
map_labels = get_CARLA_map_labels()
map_labels[(map_labels == NOT_SURE) | (map_labels == UNKNOWN)] = NOT_PLANES

In [8]:
for i, frame in enumerate(tqdm(data)):
    filename = str(i).zfill(6)

    cld = np.zeros((frame.cld.shape[0], 4))
    cld[:, :3] = frame.cld
    cld.astype(np.float32).tofile(os.path.join(data_dir, cld_dir, filename + '.bin'))

    lbl = map_labels[frame.label]
    np.save(os.path.join(data_dir, label_dir, filename), lbl.astype(np.int32))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2421/2421 [01:04<00:00, 37.66it/s]


**check**

In [27]:
cld_file = os.path.join(data_dir, cld_dir, filename + '.bin')
label_file = cld_file.replace('velodyne', 'labels').replace('.bin', '.npy')

cld = np.fromfile(cld_file, dtype=np.float32).reshape(-1, 4)
label = np.load(label_file).astype(np.int32)

In [28]:
np.allclose(cld[:, :3], data[0].cld)

True

In [29]:
np.unique(label)

array([0, 1, 2], dtype=int32)